<div align="center">
<a target="_blank" href="https://colab.research.google.com/github/sushankgghimire/GenAI-360/blob/main/1.%20Langchain%20and%20Vector%20Databases/3.%20Learning%20How%20to%20Prompt/6.%20News%20Article%20SummarizerV2/News_Article_SummarizerV2.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>
</div>

In [ ]:
from IPython.display import clear_output

In [20]:
!pip install langchain newspaper3k langchain-community requests langchain_mistralai llama-cpp-python -q
clear_output()
!pip list

Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
accelerate                       0.32.1
aiohappyeyeballs                 2.4.0
aiohttp                          3.10.5
aiosignal                        1.3.1
alabaster                        0.7.16
albucore                         0.0.13
albumentations                   1.4.14
altair                           4.2.2
annotated-types                  0.7.0
anyio                            3.7.1
argon2-cffi                      23.1.0
argon2-cffi-bindings             21.2.0
array_record                     0.5.1
arviz                            0.18.0
asn1crypto                       1.5.1
astropy                          6.1.2
astropy-iers-data                0.2024.8.26.0.31.57
astunparse                       1.6.3
async-timeout                    4.0.3
atpublic                         4.1.0
attrs                            24.2.0
audioread              

In [ ]:
import os
import json
from google.colab import userdata
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = userdata.get('LANGCHAIN_API_KEY')
os.environ["MISTRAL_API_KEY"] = userdata.get('MISTRAL_API')
os.environ["HUGGINGFACEHUB_API_TOKEN"] = userdata.get('HF_TOKEN')
os.environ["ACTIVELOOP_TOKEN"] = userdata.get('ACTIVELOOP_API')

In [ ]:
import requests
from newspaper import Article

headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'
}

article_url = "https://www.artificialintelligence-news.com/2022/01/25/meta-claims-new-ai-supercomputer-will-set-records/"

session = requests.Session()

try:
  response = session.get(article_url, headers=headers, timeout=10)

  if response.status_code == 200:
    article = Article(article_url)
    article.download()
    article.parse()

    print(f"Title: {article.title}")
    print(f"Text: {article.text}")
  else:
    print(f"Failed.")
except Exception as e:
  print(f"Errors: {e}")

Title: Meta claims its new AI supercomputer will set records
Text: Ryan Daws is a senior editor at TechForge Media with over a decade of experience in crafting compelling narratives and making complex topics accessible. His articles and interviews with industry leaders have earned him recognition as a key influencer by organisations like Onalytica. Under his leadership, publications have been praised by analyst firms such as Forrester for their excellence and performance. Connect with him on X (@gadget_ry) or Mastodon (@gadgetry@techhub.social)

Meta (formerly Facebook) has unveiled an AI supercomputer that it claims will be the world’s fastest.

The supercomputer is called the AI Research SuperCluster (RSC) and is yet to be fully complete. However, Meta’s researchers have already begun using it for training large natural language processing (NLP) and computer vision models.

RSC is set to be fully built in mid-2022. Meta says that it will be the fastest in the world once complete and 

In [ ]:
from langchain.schema import HumanMessage

article_title = article.title
article_text = article.text

template = """
As an advanced AI, you've been tasked to summarize online articles into bulleted points. Here are a few examples of how you've done this in the past:

Example 1:
Original Article: 'The Effects of Climate Change
Summary:
- Climate change is causing a rise in global temperatures.
- This leads to melting ice caps and rising sea levels.
- Resulting in more frequent and severe weather conditions.

Example 2:
Original Article: 'The Evolution of Artificial Intelligence
Summary:
- Artificial Intelligence (AI) has developed significantly over the past decade.
- AI is now used in multiple fields such as healthcare, finance, and transportation.
- The future of AI is promising but requires careful regulation.

Now, here's the article you need to summarize:

==================
Title: {article_title}

{article_text}
==================

Please provide a summarized version of the article in a bulleted list format.
"""

prompt = template.format(article_title = article_title, article_text = article_text)

messages = [HumanMessage(content=prompt)]

In [ ]:
!wget https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q8_0.gguf

--2024-09-01 02:12:16--  https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q8_0.gguf
Resolving huggingface.co (huggingface.co)... 18.238.136.112, 18.238.136.32, 18.238.136.71, ...
Connecting to huggingface.co (huggingface.co)|18.238.136.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/46/12/46124cd8d4788fd8e0879883abfc473f247664b987955cc98a08658f7df6b826/ab634d1d552dc60533e486cbcc73ad9b01358994dabf2453230e7fcd77308dc8?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27mistral-7b-instruct-v0.1.Q8_0.gguf%3B+filename%3D%22mistral-7b-instruct-v0.1.Q8_0.gguf%22%3B&Expires=1725415936&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyNTQxNTkzNn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy80Ni8xMi80NjEyNGNkOGQ0Nzg4ZmQ4ZTA4Nzk4ODNhYmZjNDczZjI0NzY2NGI5ODc5NTVjYzk4YTA4NjU4ZjdkZjZiODI2L2FiNjM0ZDFkNTUyZGM2

In [ ]:
from langchain_core.callbacks.manager import CallbackManager, StdOutCallbackHandler
from langchain_community.chat_models import ChatLlamaCpp

callback_manager = CallbackManager([StdOutCallbackHandler])

chat_model = ChatLlamaCpp(
    model_path='/content/mistral-7b-instruct-v0.1.Q8_0.gguf',
    temperature = 0,
    callback_manager = callback_manager,
    n_ctx=10000,
    repeat_penalty=1.5,
    verbose=True
)

<ipython-input-8-6b0109599ec2>:6: DeprecationWarning: callback_manager is deprecated. Please use callbacks instead.
  chat_model = ChatLlamaCpp(
llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from /content/mistral-7b-instruct-v0.1.Q8_0.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336


In [ ]:
summary = chat_model.invoke(messages)
summary


llama_print_timings:        load time =    3383.07 ms
llama_print_timings:      sample time =     140.32 ms /   200 runs   (    0.70 ms per token,  1425.30 tokens per second)
llama_print_timings: prompt eval time =  401662.85 ms /   900 tokens (  446.29 ms per token,     2.24 tokens per second)
llama_print_timings:        eval time =  192572.94 ms /   199 runs   (  967.70 ms per token,     1.03 tokens per second)
llama_print_timings:       total time =  594939.94 ms /  1099 tokens


AIMessage(content=' * Meta has unveiled an AI supercomputer called the AI Research SuperCluster (RSC) that claims to be faster than any other current system and will set new records once complete.\r\n* RSC is designed for training large natural language processing (NLP) and computer vision models, with a goal of being capable of handling trillions of parameters in future applications such as real-time voice translations or AR games.\r\n* Meta expects the supercomputer to be 20x faster than its current V100-based clusters for production use. It is also estimated to be 9x and 3x times faster at running NVIDIA Collective Communication Library (NCCL) and training large scale NLP workflows, respectively.\r\n* RSC was designed with security and privacy controls in mind so that Meta can leverage real-world examples from its production systems for research purposes such as identifying harmful content on their platforms using actual data.', response_metadata={'finish_reason': 'stop'}, id='run-6

In [ ]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import validator, BaseModel, Field, field_validator
from typing import List

class ArticleSummary(BaseModel):
  title: str = Field(description="Title of the article")
  summary: List[str] = Field(description="Bulleted list summary of the article")

  @field_validator('summary')
  def has_three_or_more_lines(cls, list_of_lines):
        if len(list_of_lines) < 3:
            raise ValueError("Generated summary has less than three bullet points!")
        return list_of_lines

parser = PydanticOutputParser(pydantic_object = ArticleSummary)

In [17]:
from langchain.prompts import PromptTemplate


# create prompt template
# notice that we are specifying the "partial_variables" parameter
template = """
You are a very good assistant that summarizes online articles.

Here's the article you want to summarize.

==================
Title: {article_title}

{article_text}
==================

{format_instructions}
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["article_title", "article_text"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

# Format the prompt using the article title and text obtained from scraping
formatted_prompt = prompt.format(article_title=article_title, article_text=article_text)

messages = [HumanMessage(content=prompt)]

In [21]:
# Trying Mistral from API beacause the one from llamacpp won't work
from langchain_mistralai.chat_models import ChatMistralAI

llm = ChatMistralAI(
    temperature=0
)

In [22]:
input = {
    'article_title': article_title,
    'article_text': article_text
}

chain = prompt | llm | parser
output = chain.invoke(input)
output

ArticleSummary(title='Meta claims its new AI supercomputer will set records', summary=["Meta (Facebook) has announced its AI Research SuperCluster (RSC), which it claims will be the world's fastest AI supercomputer once completed in mid-2022.", 'The RSC is currently used for training large NLP and computer vision models, with the aim of training models with trillions of parameters.', 'Meta hopes RSC will enable real-time voice translations for large groups speaking different languages and contribute to building the metaverse.', "RSC is expected to be 20x faster than Meta's current V100-based clusters, 9x faster at running NCCL, and 3x faster at training large-scale NLP workflows.", 'A model with tens of billions of parameters can finish training in three weeks compared to nine weeks prior to RSC.', "Meta's previous AI research infrastructure relied on open-source and publicly-available datasets; RSC is designed with security and privacy controls for using real-world examples from produ

It does work but with a different approach to call it

In [23]:
llamacpp_chain = chat_model.invoke(messages)

Llama.generate: 7 prefix-match hit, remaining 893 prompt tokens to eval

llama_print_timings:        load time =    3383.07 ms
llama_print_timings:      sample time =     140.06 ms /   200 runs   (    0.70 ms per token,  1427.93 tokens per second)
llama_print_timings: prompt eval time =  441359.89 ms /   893 tokens (  494.24 ms per token,     2.02 tokens per second)
llama_print_timings:        eval time =  187002.99 ms /   199 runs   (  939.71 ms per token,     1.06 tokens per second)
llama_print_timings:       total time =  628856.94 ms /  1092 tokens


In [24]:
llamacpp_chain

AIMessage(content=' * Meta has unveiled an AI supercomputer called the AI Research SuperCluster (RSC) that claims to be faster than any other current system and will set new records once complete.\r\n* RSC is designed for training large natural language processing (NLP) and computer vision models, with a goal of being capable of handling trillions of parameters in future applications such as real-time voice translations or AR games.\r\n* Meta expects the supercomputer to be 20x faster than its current V100-based clusters for production use. It is also estimated to be 9x and 3x times faster at running NVIDIA Collective Communication Library (NCCL) and training large scale NLP workflows, respectively.\r\n* RSC was designed with security and privacy controls in mind so that Meta can leverage real-world examples from its production systems for research purposes such as identifying harmful content on their platforms using actual data.', response_metadata={'finish_reason': 'stop'}, id='run-6